# 1. Importar tablas

In [1]:
timeorigin: .z.t

In [2]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [3]:
/ 1.2. Importar tabla remuneraciones
remunerations:get `:data/remunerations;

/show employees;

In [4]:
/ 1.3. Importar tabla de bases mínimas y máximas
minMaxBasis:("DDFFI";enlist csv) 0: `$":data/minMaxBasis.csv";

/show minMaxBasis

# 2. Variables globales

In [5]:
argname:`commonBase,
        `profBase,
        `hExtraFMBase,
        `hExtraBase,
        `irpfBase,
        `meiBase;

argrems:(`sb`cs`ge`se; 
         `sb`cs`hefm`her`ge`se;
         `hefm;
         `her;
         `sb`cs`hefm`her`ge`se`pns;
         `sb`cs`ge`se);

argdate:0Nm,
        0Nm,
        0Nm,
        0Nm,
        0Nm,
        2023.01m;

# 3. Funciones


In [6]:
getremsum:{[name;lsim;sd]:() xkey select baseType:name, sum amount     /(get) (rem)uneration (sum)
                                  by nif, month 
                                  from remunerations 
                                  where remunerationType in lsim, month>=sd;}

minmax:{[n;cg;mo;bt]                                                   /values between (min)imumm & (max)imum
    if[bt=`profBase; cg:4];
    first exec maxBase & n | minBase 
    from minMaxBasis
    where contrGroup= 4&cg, mo within(startDate;endDate)};

# 4. Generación de las tablas

In [7]:
contributionBasis: (raze getremsum'[argname;argrems;argdate]) 
                    lj `nif xkey select contrGroup, nif from employees;

update amount: minmax'[amount;contrGroup;month;baseType] 
  from `contributionBasis 
  where baseType in `commonBase`profBase`meiBase;

delete contrGroup 
  from `contributionBasis;

In [8]:
timeend: .z.t

Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [9]:
show 0^exec `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase#(baseType!amount) by nif,month from contributionBasis;

nif       month  | commonBase profBase hExtraFMBase hExtraBase irpfBase meiBa..
-----------------| ----------------------------------------------------------..
00193031S 2023.01| 1921.69    1921.69  0            0          1921.69  1921...
00193031S 2023.02| 2524.371   2524.371 0            0          2524.371 2524...
00193031S 2023.03| 2481.689   2481.689 0            0          2481.689 2481...
00193031S 2023.04| 2542.129   2651.078 0            108.9484   2651.078 2542...
00193031S 2023.05| 2537.447   2619.008 0            81.56081   2619.008 2537...
00193031S 2023.06| 2509.009   2562.774 0            53.76449   2562.774 2509...
00193031S 2023.07| 1260       1260     0            18.25154   358.9469 1260 ..
00193031S 2024.01| 2067.462   2067.462 0            0          2067.462 2067...
00193031S 2024.02| 2629.741   2629.741 0            0          2629.741 2629...
00193031S 2024.03| 2857.66    2857.66  0            0          2857.66  2857...
00193031S 2024.04| 2782.258   2782.258 0

# 5. Guardar datos en csv

In [10]:
`:data/contributionBasis set contributionBasis;

In [11]:
"Number of contribution basis: ", string count contributionBasis

"Total time spent: ",string timeend-timeorigin

"Number of contribution basis: 225504"


"Total time spent: 00:00:00.443"
